In [1]:
from socket import *
import os
import math

TFTP_PORT = 69
TFTP_BLOCK_SIZE = 512
MAX_UDP_PACKET_SIZE = 65536

In [2]:
# Helper functions go here
def get_file_block_count(filename):
    """
    Determines the number of TFTP blocks for the given file
    :param filename: THe name of the file
    :return: The number of TFTP blocks for the file or -1 if the file does not exist
    """
    try:
        # Use the OS call to get the file size
        #   This function throws an exception if the file doesn't exist
        file_size = os.stat(filename).st_size
        return math.ceil(file_size / TFTP_BLOCK_SIZE)
    except:
        return -1


def get_file_block(filename, block_number):
    """
    Get the file block data for the given file and block number
    :param filename: The name of the file to read
    :param block_number: The block number (1 based)
    :return: The data contents (as a bytes object) of the file block
    """
    file = open(filename, 'rb')
    block_byte_offset = (block_number - 1) * TFTP_BLOCK_SIZE
    file.seek(block_byte_offset)
    block_data = file.read(TFTP_BLOCK_SIZE)
    file.close()
    return block_data


def put_file_block(filename, block_data, block_number):
    """
    Writes a block of data to the given file
    :param filename: The name of the file to save the block to
    :param block_data: The bytes object containing the block data
    :param block_number: The block number (1 based)
    :return: Nothing
    """
    file = open(filename, 'wb')
    block_byte_offset = (block_number - 1) * TFTP_BLOCK_SIZE
    file.seek(block_byte_offset)
    file.write(block_data)
    file.close()


def socket_setup():
    """
    Sets up a UDP socket to listen on the TFTP port
    :return: The created socket
    """
    s = socket(AF_INET, SOCK_DGRAM)
    s.bind(('', TFTP_PORT))
    return s

In [3]:
import time
def makeRRQ(filename, numBlocks, client_socket, addr_in):
    opCode = b'\x00\x03'
    counter = 1
    addr = addr_in
    while counter <= numBlocks:
        data = get_file_block(filename, counter)
        
        msg = opCode + counter.to_bytes(2, "big") + data
        print("What follows is the message that is being SENT: ")
        print(msg, "\n\n")
        client_socket.sendto(msg, addr)
        
        
        #time.sleep(5)
        ack, addr = client_socket.recvfrom(MAX_UDP_PACKET_SIZE)
        print("What follows is the message that is being RECEIVED from", addr, ": ")
        print(ack, "\n\n")
        ackCode = ack[:2]
        if ackCode != b'\x00\x04':
            break
        else:
            blockNum = ack[2:]
            blockNum = int.from_bytes(blockNum, "big")
            if blockNum == counter:
                print("ackCode: ", ackCode)
                print("blockNum: ", blockNum)
                #print("this is supposed to be the ack" + ack[2:].decode())
                counter = counter + 1
            else:
                counter = blockNum

    
    #client_socket.sendto(b'\x00\x03'+counter.to_bytes(2, 'big'), addr)

        
        

In [4]:
def start_server():
    rec_socket = socket_setup()
    data, addr = rec_socket.recvfrom(MAX_UDP_PACKET_SIZE)
    rec_socket.close()
    
    client_socket = socket(AF_INET, SOCK_DGRAM)
    
    opcode = int.from_bytes(data[:2],"big")
    print("opcode: ", opcode)
    otherInfo = data[2:]
    fileName = otherInfo.split(b"\x00")[0]                 
    print(fileName)
    mode = otherInfo.split(b"\x00")[1]   
    numBlocks = get_file_block_count(fileName)
    if numBlocks == -1:
        errorOp = b'\x00\x05'
        errorCode = b'\x00\x01'
        errorMSG = b"File was not found"
        errorEnd = b'\x00'
        print("we were here and we sent an error i believe")
        msg = errorOp + errorCode + errorMSG + errorEnd
        client_socket.sendto(msg, addr)
    
    if opcode < 1 or opcode > 5:
        errorOp = b'\x00\x05'
        errorCode = b'\x00\x04'
        errorMSG = b"Op code is invalid"
        errorEnd = b'\x00'
        msg = errorOp + errorCode + errorMSG + errorEnd
        client_socket.sendto(msg, addr)
    
    
    if opcode == 1:
        makeRRQ(fileName, numBlocks, client_socket, addr)
    elif opcode == 2:
        pass
    elif opcode == 3:
        pass
    elif opcode == 4:
        pass
    elif opcode == 5:
        pass
    
        
    client_socket.close()
    
    # read from udp socket for a client request; use recvfrom which returns a tuple: (data, (host, port))
    # initiliaze a udp socket for sending data
    # while not done sending blocks or no errors
        # send block
        # get ACK (if the ack is not for the correct block number you may need to resend a previous block)
    #close your sockets

In [5]:
start_server()

opcode:  1
b'myfile.txt'
What follows is the message that is being SENT: 
b'\x00\x03\x00\x01hello I am kevin and I am a txt file.aieurbpiurewqbvqvlibvlisabf kjvljhe vrlj;ljFEHEABVRL IWVLIAEBGRLIVUBRLIBRVKJ HALRVH EWAV AELIU VKUAE HRVLKU REALHV wlre vkjea vkfvlbhrbvkjbvjbdskjbfdskjbfkjdsbkjsksdjj djdfheufbccjbkdsa skcskdnk oiufnkjndvc skdjbvksjdbv ksjdfkafhwkuehfiwuhfkjds kafkudsjfksjdbfksjdfnlkjanf kjdgekngfksjdfhoalfij oirejldjfn orijdf oeifj woeijf oweifjoaildnfowlkngeorejfosildjf aepsojf weosifowlienfocwuhsdnfondslkfjnsoildjfn woeihfn woirhgniuvenjsk,jfdznladknfoiwuengviurfjsvnoisdn foiwejo iwj' 


What follows is the message that is being RECEIVED from ('192.168.1.181', 55087) : 
b'\x00\x04\x00\x01' 


ackCode:  b'\x00\x04'
blockNum:  1
What follows is the message that is being SENT: 
b'\x00\x03\x00\x02esfd owiunesigvnsdzlifj wiurshgn isundv osaijdf owisgoiusndfoiajofi jsodifj oieafjoiesjfosijnvoiwejfdopiwejr oiwejfoijef wieljfoisnfdocwnme fclkjw cdwiuedfjnosienfdoisahj fowisejdfoi